# Import Libraries

In [1]:
import numpy as np
import torch

# Dataset

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [3]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

## Convert into Tensor

In [4]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


# Linear Regression Model

In [5]:
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.3331, -0.2642,  2.0860],
        [-0.7714, -0.7848, -0.3603]], requires_grad=True)
tensor([ 1.6608, -0.3144], requires_grad=True)


In [6]:
def model(x):
  return x @ w.t() + b

In [8]:
preds = model(inputs)
print(preds)
print(targets)

tensor([[  49.3417, -124.7044],
        [  81.6040, -162.6381],
        [  58.2688, -193.4916],
        [  33.5051, -126.0779],
        [  99.3353, -154.1076]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Calculate Loss

In [11]:
def mse(t1, t2):
  diff = t1 - t2
  loss = torch.sum(diff * diff) / diff.numel()
  return loss

In [12]:
loss = mse(targets, preds)
print(loss)

tensor(31907.2539, grad_fn=<DivBackward0>)


# Compute Gradient

In [13]:
loss.backward()

In [14]:
print(w)
print(w.grad)

tensor([[-0.3331, -0.2642,  2.0860],
        [-0.7714, -0.7848, -0.3603]], requires_grad=True)
tensor([[  -958.8120,  -1677.6071,   -720.1811],
        [-20417.5254, -22645.1797, -13866.6104]])


In [15]:
print(b)
print(b.grad)

tensor([ 1.6608, -0.3144], requires_grad=True)
tensor([ -11.7890, -244.2039])


In [16]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


# Train the Model using Gradient Descent

## 1. Generate predictions

In [17]:
preds = model(inputs)
print(preds)

tensor([[  49.3417, -124.7044],
        [  81.6040, -162.6381],
        [  58.2688, -193.4916],
        [  33.5051, -126.0779],
        [  99.3353, -154.1076]], grad_fn=<AddBackward0>)


## 2. Calculate the Loss

In [18]:
loss = mse(preds, targets)
print(loss)

tensor(31907.2539, grad_fn=<DivBackward0>)


## 3. Compute Gradient wrt weights & biases

In [19]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[  -958.8120,  -1677.6071,   -720.1811],
        [-20417.5254, -22645.1797, -13866.6104]])
tensor([ -11.7890, -244.2039])


## 4. Adjust weights & reset gradient to zero

In [20]:
with torch.no_grad():
  w -= w.grad * 1e-5
  b -= b.grad * 1e-5
  w.grad.zero_()
  b.grad.zero_()

In [21]:
print(w)
print(b)

tensor([[-0.3235, -0.2474,  2.0932],
        [-0.5672, -0.5584, -0.2217]], requires_grad=True)
tensor([ 1.6609, -0.3120], requires_grad=True)


## Now again predict model & Calculate Loss

In [22]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(21652.6660, grad_fn=<DivBackward0>)


# Now Calculate for multiple epochs

In [23]:
learning_rate = 1e-5
num_epochs = 100

for i in range(num_epochs):
  preds = model(inputs)
  loss = mse(preds, targets)
  loss.backward()

  with torch.no_grad():
    w -= w.grad * learning_rate
    b -= b.grad * learning_rate
    w.grad.zero_()
    b.grad.zero_()

In [24]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(209.3980, grad_fn=<DivBackward0>)


In [25]:
print(preds)
print(targets)

tensor([[ 59.0275,  73.8326],
        [ 94.5212, 100.1554],
        [ 87.6976, 128.4636],
        [ 31.4069,  57.4973],
        [117.6449, 106.2301]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


# **Linear Regression using Pytorch builtin functions**

In [26]:
import torch.nn as nn

In [27]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')


In [28]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [29]:
print(inputs)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])


In [30]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])


## Create Tensor Dataset

In [33]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [32]:
train_ds = TensorDataset(inputs, targets)
train_ds[:5]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]))

In [35]:
# Create train data loader
batch_size = 5

train_dl = DataLoader(train_ds,
                      batch_size,
                      shuffle=True)

train_dl

In [38]:
train_dl.dataset[0]

(tensor([73., 67., 43.]), tensor([56., 70.]))

In [40]:
for xb, yb in train_dl:
  print(xb)
  print(yb)
  break

tensor([[ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 87., 134.,  58.],
        [ 68.,  97.,  70.],
        [ 91.,  87.,  65.]])
tensor([[118., 134.],
        [ 20.,  38.],
        [119., 133.],
        [102., 120.],
        [ 80., 102.]])


## Define Linear Model
* nn.linear

In [41]:
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.0258,  0.5713, -0.5438],
        [-0.1637,  0.4478,  0.3733]], requires_grad=True)
Parameter containing:
tensor([-0.1637, -0.3211], requires_grad=True)


In [42]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0258,  0.5713, -0.5438],
         [-0.1637,  0.4478,  0.3733]], requires_grad=True),
 Parameter containing:
 tensor([-0.1637, -0.3211], requires_grad=True)]

In [43]:
# Generate predictions
preds = model(inputs)
preds

tensor([[16.6084, 33.7833],
        [17.6493, 48.0800],
        [47.0864, 67.0926],
        [ 6.9077, 16.0493],
        [18.3902, 57.5037],
        [16.0629, 33.1718],
        [16.5343, 48.0056],
        [46.5684, 67.3023],
        [ 7.4532, 16.6607],
        [17.8207, 58.0407],
        [15.4934, 33.7088],
        [17.1038, 47.4686],
        [48.2014, 67.1671],
        [ 7.4772, 15.5122],
        [18.9357, 58.1152]], grad_fn=<AddmmBackward>)

## Loss

In [44]:
import torch.nn.functional as F

In [49]:
loss_fn = F.mse_loss

In [46]:
loss = F.mse_loss(preds, targets)
print(loss)

tensor(3077.3845, grad_fn=<MseLossBackward>)


## Optimier

In [47]:
opt = torch.optim.SGD(model.parameters(),
                      lr=1e-5)

## Train the Model

In [50]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [51]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 112.1764
Epoch [20/100], Loss: 153.1743
Epoch [30/100], Loss: 14.1727
Epoch [40/100], Loss: 82.2482
Epoch [50/100], Loss: 83.1015
Epoch [60/100], Loss: 40.7211
Epoch [70/100], Loss: 80.5890
Epoch [80/100], Loss: 51.7061
Epoch [90/100], Loss: 43.5446
Epoch [100/100], Loss: 48.1621


In [52]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.3157,  71.0072],
        [ 76.9853,  99.0810],
        [127.9470, 135.4511],
        [ 27.9347,  41.1753],
        [ 88.8611, 113.8824],
        [ 57.0686,  69.9629],
        [ 75.8400,  98.8623],
        [127.7269, 135.9035],
        [ 29.1818,  42.2197],
        [ 88.9629, 114.7080],
        [ 57.1704,  70.7885],
        [ 75.7382,  98.0367],
        [129.0923, 135.6698],
        [ 27.8329,  40.3497],
        [ 90.1082, 114.9268]], grad_fn=<AddmmBackward>)

In [53]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])


In [54]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[53.5900, 67.8420]], grad_fn=<AddmmBackward>)

## Reference:
* https://jovian.ai/aakashns/02-linear-regression